# Load Package

In [1]:
import pandas as pd
import numpy as np
import os
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
%matplotlib inline
plt.style.use("ggplot")
plt.rcParams['font.family'] = 'Noto Sans CJK TC'
plt.rcParams['axes.unicode_minus']=False

# Load Data

In [52]:
data_acc = pd.read_csv("道路交通事故原因傷亡統計107-110.csv", dtype='unicode')
data_light = pd.read_csv("臺南市路燈資料.csv", dtype='unicode')

data_acc['GPS經度'] = data_acc['GPS經度'].astype('float')
data_acc['GPS緯度'] = data_acc['GPS緯度'].astype('float')

data_light['X'] = data_light['X'].astype('float')
data_light['Y'] = data_light['Y'].astype('float')

In [53]:
data_acc.head(3)

,案件編號,發生日期,發生時間,GPS經度,GPS緯度,案件類別,地址類型,發生地點,24小時內死亡人數,受傷人數,天候,速限-第1當事者,道路型態,事故位置,號誌種類,事故類型及型態,肇因研判,區域,季節
0,10701AC191A0013,20180119,73010,120.221200,22.963416,交通事故,一般地址,臺南市東區生產路68號,1,0,晴,50,直路,機車優先道,無號誌,撞護欄(樁),未注意車前狀態,東區,冬天
1,10701AC191B0003,20180101,14546,120.235561,22.987356,交通事故,交叉路口,臺南市東區富農街二段西側與富農街二段12巷處,0,1,晴,40,三岔路,交叉路口內,無號誌,側撞,酒醉(後)駕駛失控,東區,冬天
2,10701AC191B0023,20180101,180023,120.233903,22.991879,交通事故,一般地址,臺南市東區中華東路1段前0公尺,0,2,晴,60,直路,機車優先道,無號誌,追撞,未保持行車安全距離,東區,冬天


In [54]:
# twd97座標轉換經緯度: https://github.com/yychen/twd97
import twd97
wgs84 = [twd97.towgs84(x, y) for x, y in zip(data_light['X'], data_light['Y'])]
# 轉換出來的格式為 (緯度, 經度)
data_light['GPS經度'] = [x[1] for x in wgs84] # 經度
data_light['GPS緯度'] = [x[0] for x in wgs84]  # 緯度

In [55]:
data_light.head(3)

,區域,路燈編號,地址,設立方式,燈泡材質,燈泡瓦數,高度,燈泡數,地點描述,地標,X,Y,GPS經度,GPS緯度
0,七股區,100000,七股區看坪里看坪5-10號左,水泥桿,LED,101,NaN,1,看坪5-10左,NaN,162475.7262,2558672.506,120.145423,23.127630
1,七股區,100001,七股區看坪里看坪5-10號左,水泥桿,LED,101,NaN,1,看坪5-10左1,NaN,162457.2718,2558698.640,120.145242,23.127865
2,七股區,100002,七股區看坪里看坪6-5號前,水泥桿,LED,101,NaN,1,看坪6-5前,NaN,162439.4180,2558710.742,120.145067,23.127974


In [56]:
data_light.shape

(89928, 14)

In [57]:
data_acc.shape

(159407, 19)

In [124]:
acc_dist = list(data_acc['區域'])
light_dist = list(data_light['區域'])


# 經緯度直線距離計算: https://www.itread01.com/p/831521.html
import math
function = lambda x, y: 6371.004 * math.acos(math.sin(y0)*math.sin(y) + math.cos(y0)*math.cos(y)*math.cos(x-x0))

light_ls = []
for i in range(data_acc.shape[0]): #data_acc.shape[0]
    if i % 10000 == 0:
        print(i)
        
    now = acc_dist[i]
    x0, y0 = data_acc['GPS經度'][i], data_acc['GPS緯度'][i]
    
    # 找到目前這個點所在區域的所有路燈
    range_ = [i for i, x in enumerate(light_dist) if x == now]
    
    # 把該點與路燈做距離計算
    light_x = list(data_light['GPS經度'][range_])
    light_y = list(data_light['GPS緯度'][range_])
    D = [function(x, y) for x, y in zip(light_x, light_y)]
    
    # 取 2 公尺作為路燈照射標準
    light_num = np.sum(np.array(D) <= 2)
    # np.where(distance < 0.0135)
    light_ls.append(light_num)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000


In [129]:
data_acc['路燈數量'] = light_ls

In [130]:
data_acc.head(3)

,案件編號,發生日期,發生時間,GPS經度,GPS緯度,案件類別,地址類型,發生地點,24小時內死亡人數,受傷人數,天候,速限-第1當事者,道路型態,事故位置,號誌種類,事故類型及型態,肇因研判,區域,季節,路燈數量
0,10701AC191A0013,20180119,73010,120.221200,22.963416,交通事故,一般地址,臺南市東區生產路68號,1,0,晴,50,直路,機車優先道,無號誌,撞護欄(樁),未注意車前狀態,東區,冬天,0
1,10701AC191B0003,20180101,14546,120.235561,22.987356,交通事故,交叉路口,臺南市東區富農街二段西側與富農街二段12巷處,0,1,晴,40,三岔路,交叉路口內,無號誌,側撞,酒醉(後)駕駛失控,東區,冬天,1
2,10701AC191B0023,20180101,180023,120.233903,22.991879,交通事故,一般地址,臺南市東區中華東路1段前0公尺,0,2,晴,60,直路,機車優先道,無號誌,追撞,未保持行車安全距離,東區,冬天,0


In [131]:
# 儲存資料
data_acc.to_csv("道路交通事故原因傷亡統計107-110_有路燈.csv", index=False, encoding='utf_8_sig')

In [132]:
data_light.to_csv("臺南市路燈資料.csv", index=False, encoding='utf_8_sig')